<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [ ]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

In [ ]:
!unzip concrete_data_week4.zip

In [ ]:
import zipfile as zf

files = zf.ZipFile("concrete_data_week4.zip",'r')
files.extractall()
files.close()

In [ ]:
After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [1]:
#  Import libraris

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
#import cv2
import tensorflow as tf

In [2]:
# Import the model VGG16
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image #required for vgg16
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

In [3]:
#Global parameters

num_classes = 2
image_resize = 224

batch_size_training = 200
batch_size_validation = 190

### Data Generator 

In [4]:
# Image Data Generator instances
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [5]:
# data for traning
train_generator = data_generator.flow_from_directory(
    'C:\\Users\\bakia\\Downloads\\concrete_data_week4\\train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [6]:
# data for validation

validation_generator = data_generator.flow_from_directory(
    'C:\\Users\\bakia\\Downloads\\concrete_data_week4\\valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')


Found 9501 images belonging to 2 classes.


### Model bulding

In [7]:
model = Sequential()

In [8]:
model.add(VGG16(
    input_shape=(224,224,3),
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))
#my_new_model.add(vgg16.VGG16(include_top=False, pooling='avg', weights=resnet_weights_path))

In [9]:
# Dense layer
model.add(Dense(num_classes, activation='softmax'))

In [10]:
# Set model not train mode inorder to use tranined state
model.layers[0].trainable = False
# compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
# Epoch
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 1

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [13]:
# Fit
fit_history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

151/151 [==============================] - ETA: 0s - loss: 0.1277 - accuracy: 0.9541 

ResourceExhaustedError:  OOM when allocating tensor with shape[190,56,56,256] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node sequential/vgg16/block3_conv1/Relu (defined at \AppData\Local\Temp\ipykernel_4316\116760021.py:2) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_test_function_1888]

Function call stack:
test_function


In [14]:
# Save model
model.save('classifier_vgg16.h5')

In [ ]:
# model= tf.keras.models.load_model('classifier_vgg16.h5')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [31]:
# Test data
test_generator = data_generator.flow_from_directory(
    'C:\\Users\\bakia\\Downloads\\concrete_data_week4\\test', #? test data and directory
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation, # 1
    class_mode='categorical', #None
    shuffle=True
)

Found 500 images belonging to 2 classes.


In [25]:
# test on vgg16


# batch_size_test = 100
steps_per_epoch_test = len(test_generator) #?
scoremodel = model.evaluate_generator(test_generator, steps_per_epoch_test)
print("Accuracy of VGG16 = ",scoremodel[1])


Accuracy of VGG16 =  0.9940000176429749


In [ ]:
# Resnet50 model
# Loading is not sucsesful to compare, there must be some thing wrong in my program configuratio?

In [95]:
#test on resnet50
# load model
#model = keras.models.load_model('path/to/location') tf.keras.models.
from tensorflow.keras.models import load_model
loaded_resnet50 = load_model('C:\\Users\\bakia\\Downloads\\classifier_resnet_model.h5')

steps_per_epoch_test = len(test_generator) #?
scoremodel = loaded_resnet50.evaluate(test_generator, steps_per_epoch_test)
print("Accuracy of Resnet50 = ",scoremodel[1])



<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [27]:
test_generator.reset()

In [32]:

prediction=model.predict_generator(test_generator, steps=len(test_generator), verbose=1)

3/3 [==============================] - 34s 11s/step


In [33]:
prediction[:5]

array([[1.1572022e-02, 9.8842794e-01],
       [7.9997932e-05, 9.9992001e-01],
       [1.5195810e-04, 9.9984801e-01],
       [9.8761058e-01, 1.2389419e-02],
       [9.8702234e-01, 1.2977628e-02]], dtype=float32)

In [53]:
#pred= model.predict_generator(validation_generator, nb_validation_samples // batch_size)
predicted_class_indices=np.argmax(prediction,axis=1)
labels=(validation_generator.class_indices)
labels2=dict((v,k) for k,v in labels.items())
predictions=[labels2[k] for k in predicted_class_indices]
# print(predicted_class_indices)
# print(labels)
# print(predictions)
predictions[0:6]

['positive', 'positive', 'positive', 'negative', 'negative', 'negative']

### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).